In [1]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

import sys
sys.path.append('..')
from utils import plot_counts_and_proportion, read_train_transaction, get_categorical_from_df

In [41]:
import numpy as np
def get_trues(Ser):
    return Ser[Ser].index

In [2]:
X , y = read_train_transaction(nrows=5000)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [5]:
get_categorical_from_df(X_train)[0]

[2, 3, 4, 5, 6, 7, 8, 9, 10, 13, 14, 44, 45, 46, 47, 48, 49, 50, 51, 52]

In [44]:
X_train[get_trues(X_train.dtypes == 'O')]

,ProductCD,card4,card6,P_emaildomain,R_emaildomain,M1,M2,M3,M4,M5,M6,M7,M8,M9
TransactionID,,,,,,,,,,,,,,
2991154,W,visa,debit,aol.com,NaN,T,T,T,NaN,NaN,F,T,T,T
2991820,W,visa,credit,yahoo.com,NaN,NaN,NaN,NaN,NaN,NaN,F,NaN,NaN,NaN
2988202,W,mastercard,debit,mail.com,NaN,NaN,NaN,NaN,NaN,NaN,T,NaN,NaN,NaN
2990759,C,visa,credit,hotmail.com,hotmail.com,NaN,NaN,NaN,M0,NaN,NaN,NaN,NaN,NaN
2987622,W,visa,debit,gmail.com,NaN,NaN,NaN,NaN,M0,T,F,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2991931,W,visa,debit,anonymous.com,NaN,T,T,F,M1,T,T,NaN,NaN,NaN
2990264,W,visa,debit,gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,T,NaN,NaN,NaN
2988653,W,mastercard,debit,yahoo.com,NaN,T,T,T,NaN,NaN,F,F,T,T


In [9]:
train_data = lgb.Dataset(X_train[~get_trues(X_train.dtypes == 'O')], label=y_train)
test_data = lgb.Dataset(X_test[~get_trues(X_train.dtypes == 'O')], label=y_test)

#basic parameter:
parameters = {
    'application': 'binary',
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'boosting_type': 'gbdt',
    'boosting': 'gbdt',
    # 'categorical_feature': get_categorical_from_df(X_train)[0],
    'learning_rate': 0.05,
    'metric': 'auc',
    'min_data': 50,
    'max_depth': 10,
    'is_unbalance': 'true',
    'num_leaves': 31,
    'feature_fraction': 0.5,
    'objective': 'binary',
    'sub_feature': .5,
    'verbose': 0
}

In [35]:
type(X_train.dtypes == 'O')

pandas.core.series.Series

In [42]:
X_train[get_trues(X_train.dtypes == 'O')]

,ProductCD,card4,card6,P_emaildomain,R_emaildomain,M1,M2,M3,M4,M5,M6,M7,M8,M9
TransactionID,,,,,,,,,,,,,,
2991154,W,visa,debit,aol.com,NaN,T,T,T,NaN,NaN,F,T,T,T
2991820,W,visa,credit,yahoo.com,NaN,NaN,NaN,NaN,NaN,NaN,F,NaN,NaN,NaN
2988202,W,mastercard,debit,mail.com,NaN,NaN,NaN,NaN,NaN,NaN,T,NaN,NaN,NaN
2990759,C,visa,credit,hotmail.com,hotmail.com,NaN,NaN,NaN,M0,NaN,NaN,NaN,NaN,NaN
2987622,W,visa,debit,gmail.com,NaN,NaN,NaN,NaN,M0,T,F,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2991931,W,visa,debit,anonymous.com,NaN,T,T,F,M1,T,T,NaN,NaN,NaN
2990264,W,visa,debit,gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,T,NaN,NaN,NaN
2988653,W,mastercard,debit,yahoo.com,NaN,T,T,T,NaN,NaN,F,F,T,T


In [20]:
X_train.shape

(3750, 392)

In [10]:
clf = lgb.train(parameters,
               train_data,
               valid_sets=test_data,
               num_boost_round=2500,
               early_stopping_rounds=100)

# clf = lgb.train(parameters, train_data, 100)

#Prediction
y_pred = clf.predict(X_test)

ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in the following fields: ProductCD, card4, card6, P_emaildomain, R_emaildomain, M1, M2, M3, M4, M5, M6, M7, M8, M9

In [ ]:
ax = lightgbm.plot_importance(clf, max_num_features=40, figsize=(15,15))
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)